In [1]:
!pip install 'markitdown[all]' ffmpeg-downloader
!pip install llama-index-llms-openai_like llama-index-embeddings-openai iprogress ipywidgets jupyter 

In [2]:
import os
from markitdown import MarkItDown

result = None
if os.path.exists("./data/SENSIBLE_DATA.xlsx") is False:
    md = MarkItDown(enable_plugins=False) # Set to True to enable plugins
    result = md.convert("./data/SENSIBLE_DATA.xlsx")
else:
    print("SENSIBLE_DATA.xlsx already exists, skipping conversion.")

/home/vule/llama-rag/notebooks/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


SENSIBLE_DATA.xlsx already exists, skipping conversion.


In [3]:
if result:
    with open("./data/result.md", "w", encoding="utf-8") as f:
        f.write(result.text_content)
else:
    print("File already exists.")

File already exists.


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

base_url = os.getenv("IONOS_BASE_URL", "http://localhost:11434")
api_key = os.getenv("IONOS_API_KEY", "your_api_key_here")

os.environ["OPENAI_API_BASE"] = base_url
os.environ["OPENAI_API_KEY"] = api_key

In [5]:
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
}

In [6]:
from llama_index.llms.openai_like import OpenAILike

llama_3_3 = 'meta-llama/Llama-3.3-70B-Instruct'

llm = OpenAILike(
    api_base=base_url,
    temperature=0,
    model=llama_3_3,
    is_chat_model=True,
    default_headers=headers,
    api_key=api_key,
    context_window=128000,  # Adjusted to a more reasonable value for Llama 3.3-70B-Instruct
)

In [7]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

embed_model = OpenAIEmbedding(
  model_name=embed_model_name,
  api_base=base_url,
  api_key=api_key,
  default_headers=headers,
  embed_batch_size=10
)

In [8]:
from llama_index.core.settings import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024 * 2

In [9]:
import time
from llama_index.core.agent.workflow import AgentStream
from IPython.display import display, Markdown

async def stream_and_time(handler):
    start = time.time()
    full_response_text = ""

    async for event in handler.stream_events():
        if isinstance(event, AgentStream):
            print(event.delta, end="", flush=True)
            full_response_text += event.delta

    end = time.time()
    
    execution_time = f"# Execution time: {end - start:.2f} seconds"
    display(Markdown(f"{execution_time}"))
    return full_response_text

In [10]:
import nest_asyncio
import os

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core import StorageContext, load_index_from_storage

nest_asyncio.apply()

documents = SimpleDirectoryReader(input_files=['./data/result.md']).load_data()

# rebuild storage context
index = None
try:
    if os.path.isdir('./index') is True:
        storage_context = StorageContext.from_defaults(persist_dir="./index")
        index = load_index_from_storage(storage_context=storage_context)
    else:
        transformations = [
            SentenceSplitter(
                chunk_size=1024 * 2,
                chunk_overlap=20,
            ),
        ]
        index = VectorStoreIndex.from_documents(documents, transformations=transformations, show_progress=True)
        index.storage_context.persist('./index')
except Exception as e:
    print(e)

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/index_store.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/index_store.json.


In [11]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)

tools = [
    QueryEngineTool(
      query_engine=query_engine,
        metadata=ToolMetadata(
            name="query_tool",
            description="A tool that is Useful when you want to query through the documents"
        )
    ),
]

In [12]:
system_prompt = """ \n
You are a smart assistant designed to analyze complex insurance-related product data from an Excel file. 
You support the user by understanding data structures, modeling them, and translating them into suitable formats for stakeholders like domain experts, database admins, or frontend developers.

The Excel file includes structured extracts of insurance product configurations. 
Each contract component (e.g. savings part, risk module, or additional coverage) is modeled as a separate entity.
These groups are represented as attribute bundles with validity constraints (temporal and/or logical).

Your job is to reason step-by-step through user queries, potentially using tools in a chain of thought manner to:
- Understand the schema
- Identify entities and relationships
- Recommend modeling approaches (relational, NoSQL, frontend structure, etc.)
- Rephrase for different target audiences

## Tools

You have access to a set of specialized tools that help you analyze, 
extract, and process information effectively.
Use them wisely — not everything needs a tool, but they can help with complex or data-heavy tasks.

When a request is made, ask yourself:
- What do I need to figure out?
- Can I reason through it myself, or do I need to use a tool to get the answer?

If it makes sense to use a tool, break the task down clearly.
Choose the most suitable tool and provide it with clean, focused input. 
Once you get the result, interpret it and decide if anything else is needed.

## Output Format
Please answer in the same language as the user's input.
Think out loud before taking any action. This helps others understand your reasoning.

Repeat the thought → action → observation loop until you have enough to respond.

### When using a tool, follow this format:
Thought: [What you’re thinking and why you need the tool]
Action: [Tool name] (choose from {tool_names})
Action Input: [Tool input in JSON]
Observation: [Result you got from the tool]

### When you're done:
Thought: I have everything I need now.
Answer: [Your final answer here]

If you cannot answer:
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here – same language as user]
"""
print(system_prompt)

 

You are a smart assistant designed to analyze complex insurance-related product data from an Excel file. 
You support the user by understanding data structures, modeling them, and translating them into suitable formats for stakeholders like domain experts, database admins, or frontend developers.

The Excel file includes structured extracts of insurance product configurations. 
Each contract component (e.g. savings part, risk module, or additional coverage) is modeled as a separate entity.
These groups are represented as attribute bundles with validity constraints (temporal and/or logical).

Your job is to reason step-by-step through user queries, potentially using tools in a chain of thought manner to:
- Understand the schema
- Identify entities and relationships
- Recommend modeling approaches (relational, NoSQL, frontend structure, etc.)
- Rephrase for different target audiences

## Tools

You have access to a set of specialized tools that help you analyze, 
extract, and process 

In [13]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.core.memory import ChatMemoryBuffer

import nest_asyncio

nest_asyncio.apply()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=128.000,
    llm=llm,
)

agent = ReActAgent(
    tools=tools,
    llm=llm,
    system_prompt=system_prompt,
    max_iterations=3,
    max_execution_time=60,
    verbose=True,
)
chat_memory.reset()

ctx = Context(agent)

In [14]:
query = "Was sind das für Daten? Kannst du mir etwas modellieren?"
handler = agent.run(query, ctx=ctx, memory=chat_memory)
full_text_response = await stream_and_time(handler)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Thought: The current languageThought: The current language of the user is: German. of the user is: German. I need to use a I need to use a tool to help me answer tool to help me answer the the question.
Action question.
Action: query_tool
: query_tool
Action InputAction Input: {"input": ": {"input": "Was sind dasWas sind das f\u00fcr Daten f\u00fcr Daten? Kannst du mir etwas modell? Kannst du mir etwas modellieren?"}Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
ieren?"}Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event Ag

# Execution time: 17.32 seconds

In [15]:
%%time

import pandas as pd
from IPython.display import display

pd.set_option('display.max_colwidth', None)

# Split the response into thought and answer
thought = ""
answer = ""
if "Thought:" in full_text_response and "Answer:" in full_text_response:
    parts = full_text_response.split("Answer:", 1)
    if len(parts) > 1:
        thought = parts[0].replace("Thought:", "").strip()
        answer = parts[1].strip()

# Create DataFrame with all columns
data = {
    "Query": [query],
    "Full Response": [full_text_response],
    "Thought": [thought],
    "Answer": [answer]
}
df = pd.DataFrame(data).T

# Formatting response
display(df.style.set_properties(**{
    'white-space': 'pre-wrap',
    'text-align': 'left',
    'font-family': 'monospace',
}))

,0
Query,Was sind das für Daten? Kannst du mir etwas modellieren?
Full Response,"Thought: The current language of the user is: German. I need to use a tool to help me answer the question. Action: query_tool Action Input: {""input"": ""Was sind das f\u00fcr Daten? Kannst du mir etwas modellieren?""}Thought: Ich kann jetzt antworten, ohne weitere Tools zu verwenden. Ich werde das Modell und die Daten in deiner Sprache beschreiben. Answer: Die Daten scheinen eine Art von Versicherungsdaten zu sein, die verschiedene Arten von Versicherungen beschreiben. Ein mögliches Modell für diese Daten könnte wie folgt aussehen: Eine Versicherung hat eine eindeutige ID und eine Beschreibung, gehört zu einer bestimmten Entität und hat eine Basis-Deckung. Eine Versicherung hat verschiedene Felder, die jeweils eine bestimmte Bedeutung haben, und diese Felder haben eine bestimmte Attributbezeichnung und können einen bestimmten Wertebereich haben. Ein Beispiel für ein solches Modell könnte in einer relationalen Datenbank implementiert werden, wie im obigen Codebeispiel gezeigt."
Thought,"The current language of the user is: German. I need to use a tool to help me answer the question. Action: query_tool Action Input: {""input"": ""Was sind das f\u00fcr Daten? Kannst du mir etwas modellieren?""} Ich kann jetzt antworten, ohne weitere Tools zu verwenden. Ich werde das Modell und die Daten in deiner Sprache beschreiben."
Answer,"Die Daten scheinen eine Art von Versicherungsdaten zu sein, die verschiedene Arten von Versicherungen beschreiben. Ein mögliches Modell für diese Daten könnte wie folgt aussehen: Eine Versicherung hat eine eindeutige ID und eine Beschreibung, gehört zu einer bestimmten Entität und hat eine Basis-Deckung. Eine Versicherung hat verschiedene Felder, die jeweils eine bestimmte Bedeutung haben, und diese Felder haben eine bestimmte Attributbezeichnung und können einen bestimmten Wertebereich haben. Ein Beispiel für ein solches Modell könnte in einer relationalen Datenbank implementiert werden, wie im obigen Codebeispiel gezeigt."


CPU times: user 32.6 ms, sys: 7.28 ms, total: 39.9 ms
Wall time: 41.3 ms


In [16]:
query = "Zeig mir eine ERD-Skizze dazu oder gib mir Tabellenstruktur-Vorschläge (Name, Spalten, Relationen)."
handler = agent.run(query, ctx=ctx, memory=chat_memory)
full_text_response = await stream_and_time(handler)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Thought: Die aktuelle Sprache des BenThought: Die aktuelle Sprache des Benutzutzers ist Deutsch. Ich mussers ist Deutsch. Ich muss eine eine ERD-Skizze oder Tabellenstr ERD-Skizze oder Tabellenstruktur-Vorschläge erstellen, um demuktur-Vorschläge erstellen, um dem Benutzer zu helfen.

Um eine gena Benutzer zu helfen.

Um eine genaue ERD-Skizze oder Tabellenue ERD-Skizze oder Tabellenstrstruktur-Vorschläge zu erstellen, benötige ichuktur-Vorschläge zu erstellen, benötige ich jedoch mehr Informationen über das Thema oder jedoch mehr Informationen über das Thema oder die Anforderungen, die der Benutzer hat die Anforderungen, die der Benutzer hat. 

Action: query_tool
Action Input:. 

Action: query_tool
Action Input: {"input": "ERD-Skizze {"input": "ERD-Skizze und Tabellenstruktur-Vorschläge", und Tabellenstruktur-Vorschläge", "num_be

# Execution time: 27.12 seconds

In [17]:
%%time

import pandas as pd
from IPython.display import display

pd.set_option('display.max_colwidth', None)

# Split the response into thought and answer
thought = ""
answer = ""
if "Thought:" in full_text_response and "Answer:" in full_text_response:
    parts = full_text_response.split("Answer:", 1)
    if len(parts) > 1:
        thought = parts[0].replace("Thought:", "").strip()
        answer = parts[1].strip()

# Create DataFrame with all columns
data = {
    "Query": [query],
    "Full Response": [full_text_response],
    "Thought": [thought],
    "Answer": [answer]
}
df = pd.DataFrame(data).T

# Formatting response
display(df.style.set_properties(**{
    'white-space': 'pre-wrap',
    'text-align': 'left',
    'font-family': 'monospace',
}))

,0
Query,"Zeig mir eine ERD-Skizze dazu oder gib mir Tabellenstruktur-Vorschläge (Name, Spalten, Relationen)."
Full Response,"Thought: Die aktuelle Sprache des Benutzers ist Deutsch. Ich muss eine ERD-Skizze oder Tabellenstruktur-Vorschläge erstellen, um dem Benutzer zu helfen. Um eine genaue ERD-Skizze oder Tabellenstruktur-Vorschläge zu erstellen, benötige ich jedoch mehr Informationen über das Thema oder die Anforderungen, die der Benutzer hat. Action: query_tool Action Input: {""input"": ""ERD-Skizze und Tabellenstruktur-Vorschläge"", ""num_beams"": 5}Thought: Ich habe eine mögliche ERD-Skizze und Tabellenstruktur-Vorschläge erstellt, um dem Benutzer zu helfen. Die Vorschläge basieren auf den Beobachtungen, die ich gemacht habe, und können je nach Anforderungen und Erfordernissen des Projekts variieren. Die ERD-Skizze zeigt die Beziehungen zwischen den Tabellen ""Versicherung"", ""Deckung"", ""Beitrag"" und ""Tarif"" auf. Die Tabellenstruktur-Vorschläge enthalten die Feldnamen und Beschreibungen für jede Tabelle. Ich denke, dass diese Vorschläge eine gute Grundlage für die Erstellung einer Datenbank für Versicherungen bieten könnten. Es ist jedoch wichtig, dass der Benutzer die Vorschläge überprüft und anpasst, um sicherzustellen, dass sie den Anforderungen und Erfordernissen des Projekts entsprechen. Answer: Eine mögliche ERD-Skizze und Tabellenstruktur-Vorschläge für eine Versicherungs-Datenbank könnten wie folgt aussehen: * Eine Tabelle ""Versicherung"" mit Feldern wie ""Versicherungsnummer"", ""Kunde"", ""Deckung"" und ""Beitrag"" * Eine Tabelle ""Deckung"" mit Feldern wie ""Deckungsnummer"", ""Deckungsbezeichnung"" und ""Rang"" * Eine Tabelle ""Beitrag"" mit Feldern wie ""Beitragnummer"", ""Beitragsart"" und ""Beitragshöhe"" * Eine Tabelle ""Tarif"" mit Feldern wie ""Tarifnummer"", ""Tarifbezeichnung"" und ""Tarifbeitrag"" Die Beziehungen zwischen den Tabellen könnten wie folgt aussehen: * Eine Versicherung kann mehrere Deckungen haben (1:n) * Eine Deckung kann mehrere Beiträge haben (1:n) * Ein Beitrag kann mehrere Tarife haben (1:n) Die Tabellenstruktur-Vorschläge könnten wie folgt aussehen: ``` CREATE TABLE Versicherung ( Versicherungsnummer INT PRIMARY KEY, Kunde VARCHAR(255), Deckung INT, Beitrag INT, FOREIGN KEY (Deckung) REFERENCES Deckung(Deckungsnummer), FOREIGN KEY (Beitrag) REFERENCES Beitrag(Beitragnummer) ); CREATE TABLE Deckung ( Deckungsnummer INT PRIMARY KEY, Deckungsbezeichnung VARCHAR(255), Rang INT ); CREATE TABLE Beitrag ( Beitragnummer INT PRIMARY KEY, Beitragsart VARCHAR(255), Beitragshöhe DECIMAL(10, 2) ); CREATE TABLE Tarif ( Tarifnummer INT PRIMARY KEY, Tarifbezeichnung VARCHAR(255), Tarifbeitrag DECIMAL(10, 2) ); ``` Es ist jedoch wichtig zu beachten, dass dies nur ein möglicher Vorschlag ist und die tatsächliche ERD-Skizze und Tabellenstruktur je nach Anforderungen und Erfordernissen des Projekts variieren kann."
Thought,"Die aktuelle Sprache des Benutzers ist Deutsch. Ich muss eine ERD-Skizze oder Tabellenstruktur-Vorschläge erstellen, um dem Benutzer zu helfen. Um eine genaue ERD-Skizze oder Tabellenstruktur-Vorschläge zu erstellen, benötige ich jedoch mehr Informationen über das Thema oder die Anforderungen, die der Benutzer hat. Action: query_tool Action Input: {""input"": ""ERD-Skizze und Tabellenstruktur-Vorschläge"", ""num_beams"": 5} Ich habe eine mögliche ERD-Skizze und Tabellenstruktur-Vorschläge erstellt, um dem Benutzer zu helfen. Die Vorschläge basieren auf den Beobachtungen, die ich gemacht habe, und können je nach Anforderungen und Erfordernissen des Projekts variieren. Die ERD-Skizze zeigt die Beziehungen zwischen den Tabellen ""Versicherung"", ""Deckung"", ""Beitrag"" und ""Tarif"" auf. Die Tabellenstruktur-Vorschläge enthalten die Feldnamen und Beschreibungen für jede Tabelle. Ich denke, dass diese Vorschläge eine gute Grundlage für die Erstellung einer Datenbank für Versicherungen bieten könnten. Es ist jedoch wichtig, dass der Benutzer die Vorschläge überprüft und anpasst

CPU times: user 7.88 ms, sys: 0 ns, total: 7.88 ms
Wall time: 6.98 ms


In [18]:
query = "Kannst du mir eine Vorlage machen, wie ich die Tabellen gestalten kann? Mit Tabellenname und ihre Spalten bitte! Diese sind besonders wichtig beim Systementwurf für die Entwickler::innen, die das System umsetzen."
handler = agent.run(query, ctx=ctx, memory=chat_memory)
full_text_response = await stream_and_time(handler)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Thought: Die aktuelle Sprache des BenThought: Die aktuelle Sprache des Benutzers ist Deutsch. Ich muss einutzers ist Deutsch. Ich muss ein Tool verwenden, um eine passende Tool verwenden, um eine passende Antwort zu geben, aber in Antwort zu geben, aber in diesem Fall benötige ich keine spe diesem Fall benötige ich keine spezifischen Werkzeuge, umzifischen Werkzeuge, um eine allgemeine Vorlage für Tabellennamen eine allgemeine Vorlage für Tabellennamen und Spalten zu erstellen.

Action: Ke und Spalten zu erstellen.

Action: Keinin Tool benötigt

Thought: Um eine Vor Tool benötigt

Thought: Um eine Vorlage für Tabellen zu erstellen, müssen wir dielage für Tabellen zu erstellen, müssen wir die An Anforderungen des Systems und die Beforderungen des Systems und die Bezieziehungen zwischen den Entitäten berücksichthungen zwischen den E

# Execution time: 33.09 seconds

In [19]:
%%time

import pandas as pd
from IPython.display import display

pd.set_option('display.max_colwidth', None)

# Split the response into thought and answer
thought = ""
answer = ""
if "Thought:" in full_text_response and "Answer:" in full_text_response:
    parts = full_text_response.split("Answer:", 1)
    if len(parts) > 1:
        thought = parts[0].replace("Thought:", "").strip()
        answer = parts[1].strip()

# Create DataFrame with all columns
data = {
    "Query": [query],
    "Full Response": [full_text_response],
    "Thought": [thought],
    "Answer": [answer]
}
df = pd.DataFrame(data).T

# Formatting response
display(df.style.set_properties(**{
    'white-space': 'pre-wrap',
    'text-align': 'left',
    'font-family': 'monospace',
}))

CPU times: user 8.67 ms, sys: 216 μs, total: 8.88 ms
Wall time: 8.14 ms
